# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
Cities_DF = pd.read_csv('../output_data/CityWeatherData.csv')
Cities_DF.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed
0,ushuaia,0,AR,1659921248,65,-54.8000,-68.3000,37.06,10.36
1,faanui,61,PF,1659921332,71,-16.4833,-151.7500,76.51,25.70
2,onega,7,RU,1659921332,87,63.9061,38.1404,51.39,6.49
3,butaritari,89,KI,1659921333,73,3.0707,172.7902,81.77,14.97
4,qaanaaq,6,GL,1659921163,54,77.4840,-69.3632,40.59,5.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
#configure gmaps
gmaps.configure(api_key=g_key)

#get locations and humidity
city_locations = Cities_DF[['Lat', 'Lng']].astype(float)
humidity = Cities_DF['Humidity'].astype(float)

#format layout of map
layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'}

#add heatmap
fig = gmaps.figure(map_type='HYBRID', layout=layout)

heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
FavoriteWeather = Cities_DF.loc[(Cities_DF['Max Temp'] >= 70) & (Cities_DF['Max Temp'] <= 80)
                                   & (Cities_DF['Humidity'] <= 50)]
FavoriteWeather

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed
65,chernyshkovskiy,28,RU,1659921414,32,48.4219,42.2308,71.85,6.58
126,shubarshi,39,KZ,1659921501,45,48.5872,57.1869,72.55,8.95
130,houston,2,CA,1659921306,36,54.3998,-126.6701,74.66,10.92
152,thompson,75,CA,1659921571,49,55.7435,-97.8558,73.56,12.66
154,beaverlodge,92,CA,1659921311,42,55.2166,-119.4361,72.75,17.34
165,altay,35,CN,1659921576,33,47.8667,88.1167,75.24,1.48
168,tigil,30,RU,1659921577,39,57.8000,158.6667,71.78,2.95
184,karratha,16,AU,1659921583,20,-20.7377,116.8463,71.24,7.49
222,broome,3,AU,1659921656,31,-17.9667,122.2333,76.95,6.91
300,roebourne,9,AU,1659921810,18,-20.7833,117.1333,70.92,9.55


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = FavoriteWeather
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed
0,chernyshkovskiy,28,RU,1659921414,32,48.4219,42.2308,71.85,6.58
1,shubarshi,39,KZ,1659921501,45,48.5872,57.1869,72.55,8.95
2,houston,2,CA,1659921306,36,54.3998,-126.6701,74.66,10.92
3,thompson,75,CA,1659921571,49,55.7435,-97.8558,73.56,12.66
4,beaverlodge,92,CA,1659921311,42,55.2166,-119.4361,72.75,17.34
5,altay,35,CN,1659921576,33,47.8667,88.1167,75.24,1.48
6,tigil,30,RU,1659921577,39,57.8000,158.6667,71.78,2.95
7,karratha,16,AU,1659921583,20,-20.7377,116.8463,71.24,7.49
8,broome,3,AU,1659921656,31,-17.9667,122.2333,76.95,6.91
9,roebourne,9,AU,1659921810,18,-20.7833,117.1333,70.92,9.55


In [9]:
hotel_df['Hotel Name'] = ''

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 50000,
    "rankby": "prominence",
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get variables type from df
    lat = row['Lat']
    lng = row['Lng']
    city = row['City']
    params["location"] = f'{lat},{lng}'
        
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {city} hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except:
        print(f"Missing field/result for {city}... skipping.")
        
    print("------------")

/var/folders/h9/w_5g221s6ld4mq7bd28ddhzm0000gn/T/ipykernel_3657/2008587342.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


Closest chernyshkovskiy hotel is The hotel "Eugenia".
------------
Closest shubarshi hotel is Kenkiyak Khostel.
------------
Closest houston hotel is Prestige Hudson Bay Lodge, Premier Collection.
------------
Closest thompson hotel is Best Western Thompson Hotel & Suites.
------------
Closest beaverlodge hotel is Holiday Inn & Suites Grande Prairie-Conference Ctr, an IHG Hotel.
------------
Closest altay hotel is Atour Hotel.
------------
Closest tigil hotel is Tigil'.
------------
Closest karratha hotel is Dampier Mermaid Hotel.
------------
Closest broome hotel is Mangrove Hotel.
------------
Closest roebourne hotel is Dampier Mermaid Hotel.
------------
Closest whitecourt hotel is Holiday Inn Express & Suites Whitecourt Southeast, an IHG Hotel.
------------
Closest aksay hotel is Comfort Hotel Koktem.
------------
Closest constitucion hotel is Hampton Inn by Hilton Durango.
------------
Closest herat hotel is Arg Hotel.
------------
Closest blagodatnoye hotel is Donbass Palace.
---

In [10]:
hotel_df = hotel_df[hotel_df['Hotel Name'].notna()]
hotel_df = hotel_df.rename(columns={"Latitude": "Lat", "Longitude": "Lng"})
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed,Hotel Name
0,chernyshkovskiy,28,RU,1659921414,32,48.4219,42.2308,71.85,6.58,"The hotel ""Eugenia"""
1,shubarshi,39,KZ,1659921501,45,48.5872,57.1869,72.55,8.95,Kenkiyak Khostel
2,houston,2,CA,1659921306,36,54.3998,-126.6701,74.66,10.92,"Prestige Hudson Bay Lodge, Premier Collection"
3,thompson,75,CA,1659921571,49,55.7435,-97.8558,73.56,12.66,Best Western Thompson Hotel & Suites
4,beaverlodge,92,CA,1659921311,42,55.2166,-119.4361,72.75,17.34,Holiday Inn & Suites Grande Prairie-Conference...
5,altay,35,CN,1659921576,33,47.8667,88.1167,75.24,1.48,Atour Hotel
6,tigil,30,RU,1659921577,39,57.8000,158.6667,71.78,2.95,Tigil'
7,karratha,16,AU,1659921583,20,-20.7377,116.8463,71.24,7.49,Dampier Mermaid Hotel
8,broome,3,AU,1659921656,31,-17.9667,122.2333,76.95,6.91,Mangrove Hotel
9,roebourne,9,AU,1659921810,18,-20.7833,117.1333,70.92,9.55,Dampier Mermaid Hotel


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))